In [1]:
import os
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import classification_report
from scipy.signal import find_peaks, butter, filtfilt

# Constants
SAMPLING_RATE = 500  # Hz
ECG_FOLDER = "../../../Datasets/12-lead electrocardiogram database/ECGData"
DIAGNOSTICS_FILE = "../../../Datasets/12-lead electrocardiogram database/Diagnostics.xlsx"

# Rhythm Mapping
RHYTHM_MAPPING = {
    'AFIB': 'AFIB',
    'AF': 'AFIB',
    'SVT': 'GSVT',
    'AT': 'GSVT',
    'SAAWR': 'GSVT',
    'ST': 'GSVT',
    'AVNRT': 'GSVT',
    'AVRT': 'GSVT',
    'SB': 'SB',
    'SR': 'SR',
    'SA': 'SR'
}

# Hamilton-Tompkins QRS Detection
def preprocess_ecg(ecg_signal, sampling_rate=SAMPLING_RATE):
    """Preprocess ECG data: Bandpass filter and detect R-peaks."""
    # Bandpass filter (0.5 - 50 Hz)
    lowcut = 0.5
    highcut = 50.0
    nyquist = 0.5 * sampling_rate
    low = lowcut / nyquist
    high = highcut / nyquist
    b, a = butter(1, [low, high], btype='band')
    filtered_ecg = filtfilt(b, a, ecg_signal)

    # Derivative
    derivative = np.diff(filtered_ecg)

    # Squaring
    squared = derivative ** 2

    # Moving window integration
    window_size = int(0.150 * sampling_rate)  # 150ms
    integrated = np.convolve(squared, np.ones(window_size) / window_size, mode='same')

    # Find R-peaks
    r_peaks, _ = find_peaks(integrated, distance=sampling_rate * 0.6, height=np.mean(integrated))
    return r_peaks, filtered_ecg

def extract_features(ecg_data, r_peaks):
    """Extract features from ECG data and R-peaks."""
    if len(r_peaks) > 1:
        rr_intervals = np.diff(r_peaks) / SAMPLING_RATE * 1000  # in ms
        rr_mean = np.mean(rr_intervals)
        rr_std = np.std(rr_intervals)
    else:
        rr_mean = 0
        rr_std = 0

    ventricular_rate = 60000 / rr_mean if rr_mean > 0 else 0
    qrs_count = len(r_peaks)
    p2p_amplitude = np.max(ecg_data) - np.min(ecg_data)
    signal_energy = np.sum(ecg_data**2)

    return {
        "RRMean": rr_mean,
        "RRStd": rr_std,
        "VentricularRate": ventricular_rate,
        "QRSCount": qrs_count,
        "P2PAmplitude": p2p_amplitude,
        "SignalEnergy": signal_energy
    }

# Load and preprocess data
diagnostics_df = pd.read_excel(DIAGNOSTICS_FILE)
diagnostics_df['MappedRhythm'] = diagnostics_df['Rhythm'].map(RHYTHM_MAPPING)

features = []
labels = []

for idx, row in diagnostics_df.iterrows():
    try:
        ecg_file = os.path.join(ECG_FOLDER, row['FileName'] + '.csv')  # Add .csv extension
        if os.path.exists(ecg_file):
            ecg_data = pd.read_csv(ecg_file, skiprows=1, usecols=[1]).to_numpy().flatten()
            r_peaks, filtered_ecg = preprocess_ecg(ecg_data)
            feature = extract_features(filtered_ecg, r_peaks)
            features.append(feature)
            labels.append(row['MappedRhythm'])
    except Exception as e:
        print(f"Error processing file {row['FileName']}: {e}")

# Convert to DataFrame
features_df = pd.DataFrame(features)
features_df['Label'] = labels

# Encode labels
label_encoder = LabelEncoder()
features_df['EncodedLabel'] = label_encoder.fit_transform(features_df['Label'])

# Split data
X = features_df.drop(['Label', 'EncodedLabel'], axis=1).values
y = features_df['EncodedLabel'].values

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=6950, stratify=y)

# Normalize features for MLP
X_train_norm = (X_train - np.mean(X_train, axis=0)) / np.std(X_train, axis=0)
X_test_norm = (X_test - np.mean(X_train, axis=0)) / np.std(X_train, axis=0)

# Decision Tree Classifier
dt_classifier = DecisionTreeClassifier(random_state=6950)
dt_classifier.fit(X_train, y_train)
dt_preds = dt_classifier.predict(X_test)
print("Decision Tree Classification Report:")
print(classification_report(y_test, dt_preds, target_names=label_encoder.classes_, digits=5))

# MLP Classifier
mlp_model = tf.keras.Sequential([
    tf.keras.layers.Dense(64, activation='relu', input_shape=(X_train.shape[1],)),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(len(label_encoder.classes_), activation='softmax')
])

mlp_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
mlp_model.fit(X_train_norm, y_train, epochs=200, batch_size=128, validation_split=0.2)

mlp_preds = np.argmax(mlp_model.predict(X_test_norm), axis=1)
print("MLP Classification Report:")
print(classification_report(y_test, mlp_preds, target_names=label_encoder.classes_, digits=5))

2024-12-24 13:42:49.617570: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-12-24 13:42:49.630226: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-12-24 13:42:49.633972: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-12-24 13:42:49.643593: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-12-24 13:42:50.198160: W tensorflow/compiler/tf2

Decision Tree Classification Report:
              precision    recall  f1-score   support

        AFIB       0.54      0.57      0.56       445
        GSVT       0.60      0.60      0.60       462
          SB       0.96      0.96      0.96       778
          SR       0.86      0.81      0.83       445

    accuracy                           0.77      2130
   macro avg       0.74      0.74      0.74      2130
weighted avg       0.77      0.77      0.77      2130



/home/denuvo-drm/miniconda3/envs/CompositeADLRecognition/lib/python3.10/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
I0000 00:00:1735026220.699097  600048 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
I0000 00:00:1735026220.732346  600048 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
I0000 00:00:1735026220.

Epoch 1/50


I0000 00:00:1735026221.383852  600634 service.cc:146] XLA service 0x7db24000b920 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1735026221.383876  600634 service.cc:154]   StreamExecutor device (0): NVIDIA GeForce RTX 3070, Compute Capability 8.6
2024-12-24 13:43:41.397002: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2024-12-24 13:43:41.465208: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:531] Loaded cuDNN version 8907


193/213 ━━━━━━━━━━━━━━━━━━━━ 0s 523us/step - accuracy: 0.6577 - loss: 0.9179

I0000 00:00:1735026222.040868  600634 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


213/213 ━━━━━━━━━━━━━━━━━━━━ 3s 11ms/step - accuracy: 0.6648 - loss: 0.8976 - val_accuracy: 0.7653 - val_loss: 0.5375
Epoch 2/50
213/213 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7770 - loss: 0.5273 - val_accuracy: 0.7788 - val_loss: 0.4992
Epoch 3/50
213/213 ━━━━━━━━━━━━━━━━━━━━ 0s 734us/step - accuracy: 0.7800 - loss: 0.5183 - val_accuracy: 0.7917 - val_loss: 0.4862
Epoch 4/50
213/213 ━━━━━━━━━━━━━━━━━━━━ 0s 801us/step - accuracy: 0.7923 - loss: 0.5022 - val_accuracy: 0.8005 - val_loss: 0.4755
Epoch 5/50
213/213 ━━━━━━━━━━━━━━━━━━━━ 0s 783us/step - accuracy: 0.7987 - loss: 0.4781 - val_accuracy: 0.8110 - val_loss: 0.4688
Epoch 6/50
213/213 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8009 - loss: 0.4691 - val_accuracy: 0.7934 - val_loss: 0.4667
Epoch 7/50
213/213 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7940 - loss: 0.4615 - val_accuracy: 0.7993 - val_loss: 0.4576
Epoch 8/50
213/213 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8043 - loss: 0.4502 - val_accuracy: 0.8075